In [1]:
import requests
import json
from datetime import datetime
import sqlite3
import import_ipynb
import utils 


# Define the URL for the API and headers for the request
api_url = "https://api.membersports.com/api/v1/golfclubs/onlineBookingTeeTimes"
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json; charset=UTF-8",
    "x-api-key": "A9814038-9E19-4683-B171-5A06B39147FC",
}

# Convert minutes from midnight to standard time format
def convert_minutes_to_time(minutes):
    hours = minutes // 60
    mins = minutes % 60
    period = "AM" if hours < 12 else "PM"
    hours = hours % 12
    hours = 12 if hours == 0 else hours
    return f"{hours}:{mins:02d} {period}"

# Fetch data from the API
def fetch_tee_times(api_data):
    response = requests.post(api_url, headers=headers, json=api_data)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data. Status Code: {response.status_code}, Response: {response.text}")
        return None

# Format tee times from API response data
def format_tee_times(api_data, api_response_data):
    tee_time_data = []
    for entry in api_response_data:
        for item in entry["items"]:
            tee_time = convert_minutes_to_time(int(item['teeTime']))
            tee_time_data.append((
                api_data['date'], item['golfCourseId'], item['name'], tee_time, item['minimumNumberOfPlayers'], item['golfCourseNumberOfHoles'], item['price'],
            ))
    return tee_time_data

# Main execution
def get_foothills_tee_time():
    golfClubGroupIds = [1, 3]
    all_tee_time_data = []
    date = datetime.today().strftime('%Y-%m-%d')
    
    for golfClubGroupId in golfClubGroupIds:
        api_data = {
            "configurationTypeId": 0,
            "date": date,
            "golfClubGroupId": golfClubGroupId,
            "groupSheetTypeId": 0
        }
        
        try:
            api_response_data = fetch_tee_times(api_data)
            if api_response_data:
                tee_time_data = format_tee_times(api_data, api_response_data)
                all_tee_time_data.extend(tee_time_data)
        except Exception as e:
            print(f"Error fetching tee times: {e}")
    
    if all_tee_time_data:
        utils.update_database(all_tee_time_data)
        print(f"Foothills data has been saved and updated in the SQLite database 'GolfteeTimes_database.db'.")
    else:
        print("No tee time data fetched from Foothills.")
    
    return all_tee_time_data  # Return the data for use in utils.main()

if __name__ == "__main__":
    utils.initialize_database()
    get_foothills_tee_time()

Foothills data has been saved and updated in the SQLite database 'GolfteeTimes_database.db'.
